In [1]:
class journallingClass:
    def __init__(self, journalType, title, category, dateCreated, mood, journalClass):
        self.journalType = journalType
        self.journalTitle = title
        self.journalCategory = category 
        self.dateCreated = dateCreated
        self.mood = mood
        self.journalClass = journalClass
    
    def getJournalClass(self):
        return self.journalClass
        
        
class journalEntryClass:
    def __init__(self, userEntry, harmIntent = 0, modelResponse = []):
        self.userEntry = userEntry 
        self.harmIntent = harmIntent
        self.modelResponse = modelResponse
#         self.safetyResponse = safetyResponse
    def printEntry(self):
        print(f"entry: {self.userEntry}, harmful intent: {self.harmIntent}, model response: {self.modelResponse}")

        
    def updateHarmIntent(self, harmIntent):
        self.harmIntent = harmIntent
    
    def addModelResponse(self, response):
        self.modelResponse.append(response)
        
class casualJournal(journallingClass):
    
    def __init__(self, entries = []):
        self.entries = []
        
    def createNewEntry(self, userEntry,  harmIntent = 0, modelResponse = []):
        self.entries.append(journalEntryClass(userEntry, harmIntent, modelResponse))
        return self.entries[-1]
        
    def printJournal(self):
        for entry in self.entries:
            entry.printEntry()
    
    
        
    
class catharticJournal(journallingClass):
    def __init__(self, stage = 0, entries = {}):
        self.stage = stage # default stage 0
        self.entries = entries
        
    
    def createStage(self, journalEntry):
        self.entries[f"stage{self.stage}"] = journalEntry
    
    def updateStage(self):
        if self.stage >=5:
            return 
        self.stage += 1
        
    def createNewEntry(self, userEntry,  harmIntent = 0, modelResponse = []):
        self.updateStage()
        self.entries[f"stage{self.stage}"] = journalEntryClass(userEntry, harmIntent, modelResponse)
        return self.entries[f"stage{self.stage}"]
    
    def printJournal(self):
        for stage in self.entries.keys():
            print(stage)
            self.entries[stage].printEntry()


In [13]:
def generate_text(userEntry):
    return "<Model's empathetic Response>"

def checkHarmfulIntent(journalEntry):
    # implement later
    journalEntry.updateHarmIntent(0)
    # If this is harmful update the harmful intent to be set to 1
    return False 

def buildHarmresponse():
    response = "Reach out to a therapist :)"
    return response

def generateCatharticResponse(journalEntry, journal):
    if journal.getJournalClass().stage == 5:
        response = "congitive reframing"
    else:
        response = generate_text(journalEntry.userEntry)
    return response

def generateCasualResponse(journalEntry):
    res = generate_text(journalEntry.userEntry)
    # update to res[0] later
    return res

def promptSafetyDetector(journalEntry, safety_counter):
    # remove safety_counter after testing
    if safety_counter > 3:
        return False
    return True

def generatePrompt(typeJ, journalEntry, journal):
    safety_counter = 0
    safety_threshold = 5
    
    while(safety_counter<safety_threshold):
        if typeJ == 1:
            response = generateCatharticResponse(journalEntry, journal)
        else:
            response = generateCasualResponse(journalEntry)
        # check safety
        safe = promptSafetyDetector(journalEntry, safety_counter)
        if safe:
            break
        safety_counter += 1
        
    if safety_counter == safety_threshold:
        response = "<default response - model response was unsafe.>"
    
    # add model response to current entry object
    journalEntry.addModelResponse(response)
    
    return response

In [14]:
def main(response):
    
    if response["type"] == 0: # get type check from backend
        journalChild = casualJournal()
    else:
        journalChild = catharticJournal()
        
    # pull journalType, title, category, dateCreated from backend
    journal = journallingClass(response["type"], response["title"], response["category"], response["date"], response["mood"], journalChild)
    
    # pull journal entries, loop through and load 
    for entry, harmfulIntent, modelResponse in zip(response["entry"], response["harmfulIntent"], response["modelResponse"]):
        journal.getJournalClass().createNewEntry(entry, harmfulIntent, modelResponse)
    
    # get latest entry
    currentEntry = journal.getJournalClass().createNewEntry(response["entry"][-1])
    
    currentEntry.printEntry()
    
    
    harmIntent = checkHarmfulIntent(currentEntry)
    
    if harmIntent: 
        modelResponse = buildHarmresponse() 
        currentEntry.addModelResponse(modelResponse)
        return 
    
    modelResponse = generatePrompt(response["type"], currentEntry, journal)
    
#     currentEntry.printEntry()
    journal.getJournalClass().printJournal()
    
    
    result = response
    result["modelResponse"].append(modelResponse)
    

In [15]:
response = {
    "entry": ["Jonny's cat died today, it makes me feel sad. Can you provide some empathy", "entry part 2", "entry part 3", "entry part 4"],
    "harmfulIntent": [0, 0, 0, 0],
    "modelResponse": [["response 1"], ["resp2"], ["resp3"], ["resp4"]],
    "type": 1, # assume 0 casual,
    "title": "johny's cat died", 
    "userID": 8940823, 
    "date": "14/12/2023", 
    "category": "cat died",
    "mood": "mood"
}
main(response)

entry: entry part 4, harmful intent: 0, model response: []
stage1
entry: Jonny's cat died today, it makes me feel sad. Can you provide some empathy, harmful intent: 0, model response: ['response 1']
stage2
entry: entry part 2, harmful intent: 0, model response: ['resp2']
stage3
entry: entry part 3, harmful intent: 0, model response: ['resp3']
stage4
entry: entry part 4, harmful intent: 0, model response: ['resp4']
stage5
entry: entry part 4, harmful intent: 0, model response: ['congitive reframing']


In [ ]:
response